In [1]:
import copy
from collections import namedtuple
from functools import wraps

In [2]:
# There should be 16 operations.

all_ops = []

def make_op(op):
    """
    We are a bit lazy and just make a copy of the registers everytime
    to avoid state collisions
    """
    @wraps(op)
    def new_op(regs, a, b, c):
        regs = list(copy.copy(regs))
        op(regs, a, b, c)
        return tuple(regs)
    all_ops.append(new_op)
    return new_op

@make_op
def addr(regs, a, b, c):
    regs[c] = regs[a] + regs[b]

@make_op
def addi(regs, a, b, c):
    regs[c] = regs[a] + b

@make_op
def mulr(regs, a, b, c):
    regs[c] = regs[a] * regs[b]

@make_op
def muli(regs, a, b, c):
    regs[c] = regs[a] * b

@make_op
def banr(regs, a, b, c):
    regs[c] = regs[a] & regs[b]

@make_op
def bani(regs, a, b, c):
    regs[c] = regs[a] & b

@make_op
def borr(regs, a, b, c):
    regs[c] = regs[a] | regs[b]

@make_op
def bori(regs, a, b, c):
    regs[c] = regs[a] | b

@make_op
def setr(regs, a, b, c):
    regs[c] = regs[a]

@make_op
def seti(regs, a, b, c):
    regs[c] = a

@make_op
def gtri(regs, a, b, c):
    regs[c] = 1 if regs[a] > b else 0

@make_op
def gtir(regs, a, b, c):
    regs[c] = 1 if a > regs[b] else 0

@make_op
def gtrr(regs, a, b, c):
    regs[c] = 1 if regs[a] > regs[b] else 0

@make_op
def eqri(regs, a, b, c):
    regs[c] = 1 if regs[a] == b else 0

@make_op
def eqir(regs, a, b, c):
    regs[c] = 1 if a == regs[b] else 0

@make_op
def eqrr(regs, a, b, c):
    regs[c] = 1 if regs[a] == regs[b] else 0

print('There are {} ops'.format(len(all_ops)))

There are 16 ops


In [3]:
# Read in the input file

Line = namedtuple('Line', 'before instruction after')
lines = []

with open('input.txt') as f:
    while True:
        before = next(f)
        instruction = next(f)
        after = next(f)
        blank = next(f)

        if not before.startswith('Before'):
            # The input breaks to the second half at some point
            break
            
        line = Line(before=tuple(eval(before[len('Before: '):].strip())),
             after=tuple(eval(after[len('After: '):].strip())),
             instruction=tuple(map(int, instruction.strip().split())))
        lines.append(line)
        
# Make sure we read things right
print('Number of lines:', len(lines))
print('First line:', lines[0])
print('Last line:', lines[-1])

Number of lines: 812
First line: Line(before=(1, 2, 3, 2), instruction=(3, 1, 3, 0), after=(1, 2, 3, 2))
Last line: Line(before=(1, 0, 1, 2), instruction=(1, 1, 0, 1), after=(1, 1, 1, 2))


In [4]:
# Part 1: How many can satisfy at least three instructions?
sum(1 for line in lines 
    if sum(op(line.before, 
              line.instruction[1],
              line.instruction[2],
              line.instruction[3]) == line.after
           for op in all_ops) >= 3)

592

In [5]:
# Part 2a: Deduce which code corresponds to which op.
# At first, all codes are possible

possible_codes = {
    op: set(range(16)) for op in all_ops
}

In [6]:
# But each hint reduces the number of possibibilites
for line in lines:
    possible_ops = [op for op in all_ops if op(line.before, 
              line.instruction[1],
              line.instruction[2],
              line.instruction[3]) == line.after]
    for op in set(all_ops) - set(possible_ops):
        if line.instruction[0] in possible_codes[op]:
            possible_codes[op].remove(line.instruction[0])

In [7]:
# It turns out that some ops have many possible codes!
possible_codes

{<function __main__.addr(regs, a, b, c)>: {1, 2, 7, 8, 9},
 <function __main__.addi(regs, a, b, c)>: {2, 9, 12, 14},
 <function __main__.mulr(regs, a, b, c)>: {0, 5, 6, 9, 12},
 <function __main__.muli(regs, a, b, c)>: {5, 7, 9},
 <function __main__.banr(regs, a, b, c)>: {9},
 <function __main__.bani(regs, a, b, c)>: {7, 9},
 <function __main__.borr(regs, a, b, c)>: {1, 2, 5, 6, 7, 9, 12, 14, 15},
 <function __main__.bori(regs, a, b, c)>: {2, 5, 7, 9, 12, 14, 15},
 <function __main__.setr(regs, a, b, c)>: {2, 7, 9},
 <function __main__.seti(regs, a, b, c)>: {1, 2, 3, 4, 9, 11, 13, 14},
 <function __main__.gtri(regs, a, b, c)>: {2, 4, 9, 11, 13},
 <function __main__.gtir(regs, a, b, c)>: {2, 9, 11},
 <function __main__.gtrr(regs, a, b, c)>: {2, 4, 7, 9, 11},
 <function __main__.eqri(regs, a, b, c)>: {1, 7, 11},
 <function __main__.eqir(regs, a, b, c)>: {1, 10, 11},
 <function __main__.eqrr(regs, a, b, c)>: {3, 11, 13}}

In [8]:
# But if we go step by step and assign those that only have
# one possibility

definite_codes = {}
while possible_codes:
    for op, possibles in possible_codes.items():
        # If there's only one possibility, assign to definite_codes
        if len(possibles) == 1:
            definite_codes[op] = next(iter(possibles))

    # Remove all ops from possible_codes that have been assigned
    for op in definite_codes:
        if op in possible_codes:
            del possible_codes[op]

    # Remove assigned codes from possible_codes
    for val in definite_codes.values():
        for op in possible_codes:
            if val in possible_codes[op]:
                possible_codes[op].remove(val)

In [9]:
# They all get assigned!
print('Leftover possible codes:', len(possible_codes))
print('Definite codes assigned:', len(definite_codes))

Leftover possible codes: 0
Definite codes assigned: 16


In [10]:
# Change op -> code to code -> op
code_to_op = [(v, k) for k, v in definite_codes.items()]
code_to_op.sort()
code_to_op = [k for _, k in code_to_op]

In [11]:
# Part 2b: Read in the program
program = []
with open('program.txt') as f:
    for line in f:
        line = line.strip()
        program.append(tuple(map(int, line.split())))
        
# For debugging...
print('First line of program:', program[0])
print('Last line of program:', program[-1])

First line of program: (5, 1, 0, 0)
Last line of program: (2, 3, 0, 0)


In [12]:
# Run the program, starting with 0s in every register
regs = (0, 0, 0, 0)
for instruction in program:
    regs = code_to_op[instruction[0]](regs, *instruction[1:])

In [13]:
# The answer to part 2 is the value in register 0
print(regs)

(557, 6, 0, 557)
